# Covid-19 Data Download

In [ ]:
import pandas as pd
from IPython.display import display, HTML
import io
import base64

## Load the Data

In [ ]:
covid_case_data_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
case_data = pd.read_csv(covid_case_data_url)
covid_death_data_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
death_data = pd.read_csv(covid_death_data_url)

## Collect Data of selected Countries

In [ ]:
country_sequence = ['US', 'China','United Kingdom', 'Germany', 'Italy', 'Spain', 'Russia', 'France',
                    'Switzerland', 'Austria', 'Brazil', 'India', 'Vietnam', 'World', 'Sweden', 'Israel']

countries = ['US', 'China','United Kingdom', 'Germany', 'Italy', 'Spain', 'Russia', 'France',
           'Switzerland', 'Austria', 'Brazil', 'India', 'Vietnam', 'Sweden', 'Israel']

def select_data(data_src, selected_countries):
    data = data_src.copy(deep=True)
    dat = data.iloc[:,4:]
    world = ['','World',0.0,0.0] + list(dat.sum(axis=0))
    data.loc[len(data)] = world
    days = list(data.columns[4:])
    values = {"Dates": days}
    selected_data = pd.DataFrame(values, columns = ["Dates"])
    selected_data['Days'] = pd.to_datetime(selected_data['Dates'], format='%m/%d/%y')
    selected_data.drop('Dates',axis=1,inplace=True)
    vals = data.loc[data['Country/Region'].isin(selected_countries)].drop(['Province/State','Long','Lat'],axis='columns')
    vals = vals.groupby('Country/Region').sum().transpose()
    vals.reset_index(drop=True, inplace=True)
    res = pd.concat([selected_data,vals],axis=1)
    res.set_index('Days',inplace=True)
    return res


set_of_countries = set(country_sequence)
case_sel_data = select_data(case_data, set_of_countries)
death_sel_data= select_data(death_data, set_of_countries)
last_day = case_sel_data.index[-1].strftime('%d-%b-%Y')

## Create Download-Links for the Data of the last Day for selected Countries

In [ ]:
def get_df_last_day_data(data,prefix = ''):
    last_day = data.index[-1].strftime('%d-%b-%Y')
    last_day_data_dict =  {
        f'{prefix} kumulativ {last_day}': data.iloc[-1],
        f'{prefix} pro Tag {last_day}': data.diff().iloc[-1]
    }
    last_day_data = pd.DataFrame(last_day_data_dict)
    return last_day_data


sorterIndex = dict(zip(country_sequence, range(len(country_sequence))))


result = pd.concat([get_df_last_day_data(case_sel_data,"Infizierte"),
                    get_df_last_day_data(death_sel_data,"Tote")],axis=1)
result['country_rank'] = result.index.map(sorterIndex)
result.sort_values('country_rank',inplace=True)
result.drop('country_rank', 1, inplace = True)
res = result.stack()

def create_download_link_excel( df, title = "Download Excel file", filename = "data.xls"):
    output = io.BytesIO()
    writer = pd.ExcelWriter(output, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    writer.save()
    xlsx_data = output.getvalue()
    b64 = base64.b64encode(xlsx_data)
    payload = b64.decode()
    html = '<a download="{filename}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link_excel(res, f"Download Data for {last_day}", f"data_{last_day}.xlsx")